In [9]:
!pip install haversine

In [10]:
import pandas as pd
from scipy.spatial import distance_matrix
from haversine import haversine, Unit # per calcular distancies (m) amb coordenades

In [3]:
df_trans = pd.read_csv('./Data/TRANSPORTS.csv')
global_df = pd.read_csv('./Data/global_df.csv')

In [ ]:
#global_df_test = global_df.sample(n=10000, random_state=42)
estacions_coords = df.drop_duplicates(subset=['station_id','lat','lon'])

In [6]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CODI_CAPA       684 non-null    object 
 1   CAPA_GENERICA   684 non-null    object 
 2   NOM_CAPA        684 non-null    object 
 3   ED50_COORD_X    684 non-null    float64
 4   ED50_COORD_Y    684 non-null    float64
 5   ETRS89_COORD_X  684 non-null    float64
 6   ETRS89_COORD_Y  684 non-null    float64
 7   LONGITUD        684 non-null    float64
 8   LATITUD         684 non-null    float64
 9   EQUIPAMENT      684 non-null    object 
 10  DISTRICTE       488 non-null    float64
 11  BARRI           488 non-null    float64
 12  NOM_DISTRICTE   488 non-null    object 
 13  NOM_BARRI       488 non-null    object 
 14  ADRECA          14 non-null     object 
 15  TELEFON         11 non-null     object 
dtypes: float64(8), object(8)
memory usage: 85.6+ KB


In [5]:
global_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969792 entries, 0 to 2969791
Data columns (total 16 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   station_id                  int64  
 1   year                        int64  
 2   month                       int64  
 3   day                         int64  
 4   hour                        int64  
 5   num_bikes_available         float64
 6   num_docks_available         float64
 7   lat                         float64
 8   lon                         float64
 9   capacity                    float64
 10  altitude                    float64
 11  percentage_docks_available  float64
 12  ctx-1                       float64
 13  ctx-2                       float64
 14  ctx-3                       float64
 15  ctx-4                       float64
dtypes: float64(11), int64(5)
memory usage: 362.5 MB


In [16]:
coord_estacio = global_df.loc[0,['lat','lon']]
coord_transp = df_trans.loc[0,['LATITUD','LONGITUD']]
dist = haversine(coord_estacio, coord_transp, unit=Unit.METERS)
print(f"Distancia entre estació bincing {coord_estacio} i parada transport {coord_transp} són {dist:.2f} metres")

Distancia entre estació bincing lat    41.397978
lon     2.180107
Name: 0, dtype: float64 i parada transport LATITUD     41.410259
LONGITUD     2.189397
Name: 0, dtype: object són 1570.10 metres


La distància entre aquestes dos coordenades es de ~1.500m. Fem la comprovació a google per veure que ho estem calculant correctament. El traecte amb cotxe són 2,5km i hem de tenir en compte que nosaltres estem calculant la diagonal. Per tant, sembla que el càlcul és correcte.<br>
![](clot_aeroport.png)

Per tenir un ordre de magnitut, el trajecte caminant entre l'estació de bicing i la parada de metro Monumental és de ~400m, i la distància entre les dos coordenades és de ~300m. Per tant, segurament el ratio a fer servir són aprox. 500m
<br>
![Image Description](monumental.png)

In [18]:
# 41.40065935819499, 2.179468819903669
haversine(coord_estacio, [41.40065935819499,2.179468819903669], unit=Unit.METERS)

302.8777316399031

In [28]:
estacions_coords = global_df[['station_id','lat','lon']].drop_duplicates()
estacions_coords.shape

(632, 3)

In [29]:

def count_trans_within_radius(row, est_bicing, radi):
    """
    Per cada fila del data frame compta el num d'estacions de transports
    qui hi ha en el radi definit en metres
    """
    distances = [haversine((row['lat'], row['lon']), (p['LATITUD'], p['LONGITUD']), unit=Unit.METERS)
                 for _, p in est_bicing.iterrows()]
    return len([d for d in distances if d <= radi])

estacions_coords['n_transp_500m'] = estacions_coords.apply(count_trans_within_radius, args=(df_trans, 500), axis=1)

print(estacions_coords.head())


    station_id        lat       lon  n_transp_500m
0            1  41.397978  2.180107              3
15           2  41.395488  2.177198             11
30           3  41.394156  2.181331             13
46           4  41.393317  2.181248             10
61           5  41.391103  2.180176              9


In [32]:
estacions_coords['n_transp_500m'].describe()

count    632.000000
mean       7.107595
std        5.703265
min        0.000000
25%        3.750000
50%        6.000000
75%        9.000000
max       36.000000
Name: n_transp_500m, dtype: float64

L'estació de bicing què té més parades de transport públic en un rati de 500m és a gran via. 

In [37]:
estacions_coords[estacions_coords['n_transp_500m'] == 36]

,station_id,lat,lon,n_transp_500m
4034,287,41.389063,2.167931,36


In [43]:
estacions_coords.to_csv('./Data/bicing_transports.csv')

![](gran_via.png)

In [39]:
global_df_merged = global_df.merge(estacions_coords, on=['station_id','lat','lon'], how='left')

In [42]:
global_df_merged['n_transp_500m'].describe()

count    2.969792e+06
mean     7.167215e+00
std      5.905094e+00
min      0.000000e+00
25%      3.000000e+00
50%      6.000000e+00
75%      9.000000e+00
max      3.600000e+01
Name: n_transp_500m, dtype: float64